In [1]:
from pyspark.sql import Row

In [11]:
df1 = sqlContext.createDataFrame(sc.parallelize([
    Row(id=1, c1="a1", c2="bb1", c3=10),
    Row(id=2, c1="a1", c2="bb2", c3=12),
    Row(id=3, c1="a2", c2="bb3", c3=15),
    Row(id=4, c1="a3", c2="bb4", c3=5),
    Row(id=5, c1="a4", c2="bb5", c3=7),
]))

df2 = sqlContext.createDataFrame(sc.parallelize([
    Row(id=1, cc1="cc1"),
    Row(id=3, cc1="cc3"),
    Row(id=4, cc1="cc4"),
    Row(id=6, cc1="cc6"),
]))

In [9]:
df1.show()

+---+---+---+---+
| c1| c2| c3| id|
+---+---+---+---+
| a1|bb1| 10|  1|
| a1|bb2| 12|  2|
| a2|bb3| 15|  3|
| a3|bb4|  5|  4|
| a4|bb5|  7|  5|
+---+---+---+---+



In [12]:
df2.show()

+---+---+
|cc1| id|
+---+---+
|cc1|  1|
|cc3|  3|
|cc4|  4|
|cc6|  6|
+---+---+



In [14]:
df1.registerTempTable("t1")
df2.registerTempTable("t2")

In [16]:
sqlContext.sql("SELECT * FROM t1").show()

+---+---+---+---+
| c1| c2| c3| id|
+---+---+---+---+
| a1|bb1| 10|  1|
| a1|bb2| 12|  2|
| a2|bb3| 15|  3|
| a3|bb4|  5|  4|
| a4|bb5|  7|  5|
+---+---+---+---+



In [17]:
sqlContext.sql("SELECT * FROM t2").show()

+---+---+
|cc1| id|
+---+---+
|cc1|  1|
|cc3|  3|
|cc4|  4|
|cc6|  6|
+---+---+



In [18]:
sqlContext.sql("""
SELECT * 
FROM t1
JOIN t2 ON t1.id = t2.id
""").show()

+---+---+---+---+---+---+
| c1| c2| c3| id|cc1| id|
+---+---+---+---+---+---+
| a1|bb1| 10|  1|cc1|  1|
| a2|bb3| 15|  3|cc3|  3|
| a3|bb4|  5|  4|cc4|  4|
+---+---+---+---+---+---+



In [19]:
sqlContext.sql("""
SELECT * 
FROM t1
INNER JOIN t2 ON t1.id = t2.id
""").show()

+---+---+---+---+---+---+
| c1| c2| c3| id|cc1| id|
+---+---+---+---+---+---+
| a1|bb1| 10|  1|cc1|  1|
| a2|bb3| 15|  3|cc3|  3|
| a3|bb4|  5|  4|cc4|  4|
+---+---+---+---+---+---+



In [20]:
sqlContext.sql("""
SELECT * 
FROM t1
LEFT JOIN t2 ON t1.id = t2.id
""").show()

+---+---+---+---+----+----+
| c1| c2| c3| id| cc1|  id|
+---+---+---+---+----+----+
| a1|bb1| 10|  1| cc1|   1|
| a1|bb2| 12|  2|null|null|
| a2|bb3| 15|  3| cc3|   3|
| a3|bb4|  5|  4| cc4|   4|
| a4|bb5|  7|  5|null|null|
+---+---+---+---+----+----+



In [21]:
sqlContext.sql("""
SELECT * 
FROM t1
RIGHT JOIN t2 ON t1.id = t2.id
""").show()

+----+----+----+----+---+---+
|  c1|  c2|  c3|  id|cc1| id|
+----+----+----+----+---+---+
|  a1| bb1|  10|   1|cc1|  1|
|  a2| bb3|  15|   3|cc3|  3|
|  a3| bb4|   5|   4|cc4|  4|
|null|null|null|null|cc6|  6|
+----+----+----+----+---+---+



In [22]:
sqlContext.sql("""
SELECT * 
FROM t1
FULL JOIN t2 ON t1.id = t2.id
""").show()

+----+----+----+----+----+----+
|  c1|  c2|  c3|  id| cc1|  id|
+----+----+----+----+----+----+
|  a1| bb1|  10|   1| cc1|   1|
|  a1| bb2|  12|   2|null|null|
|  a2| bb3|  15|   3| cc3|   3|
|  a3| bb4|   5|   4| cc4|   4|
|  a4| bb5|   7|   5|null|null|
|null|null|null|null| cc6|   6|
+----+----+----+----+----+----+



In [23]:
df1.show()

+---+---+---+---+
| c1| c2| c3| id|
+---+---+---+---+
| a1|bb1| 10|  1|
| a1|bb2| 12|  2|
| a2|bb3| 15|  3|
| a3|bb4|  5|  4|
| a4|bb5|  7|  5|
+---+---+---+---+



In [24]:
sqlContext.sql("""
SELECT c1, SUM(c3) as sum_c3
FROM t1
GROUP BY c1
""").show()

+---+------+
| c1|sum_c3|
+---+------+
| a1|    22|
| a2|    15|
| a3|     5|
| a4|     7|
+---+------+



kodėl įvyks klaida?

In [25]:
sqlContext.sql("""
SELECT c1, c2, MAX(c3) as sum_c3
FROM t1
GROUP BY c1
""").show()

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



AnalysisException: expression 'c2' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() if you don't care which value you get.;

In [30]:
maxByC2RDD = df1.rdd.map(lambda r: (r.c1, r)).reduceByKey(lambda r1, r2: r1 if r1.c3 > r2.c3 else r2).values()
maxByC2RDD.collect()

[Row(c1='a1', c2='bb2', c3=12, id=2),
 Row(c1='a3', c2='bb4', c3=5, id=4),
 Row(c1='a4', c2='bb5', c3=7, id=5),
 Row(c1='a2', c2='bb3', c3=15, id=3)]